In [9]:
import pandas as pd
import numpy  as np

Task 1: Create Import file (try, except)

In [10]:
print("Enter a class to grade (i.e. class1 for class1)")
name = input()
try:
    with open(name+'.txt','r') as gs:
        grades = gs.readlines()
        print("Successfully opened", name+'.txt')
        #print(grades)
except IOError:
        print("File not accessible")

Enter a class to grade (i.e. class1 for class1)
Successfully opened class3.txt


Task 2: Inspect Value (check if has 26 values, print invalid, report)

for loop to check if has 26 values, if not, print

In [11]:
#Make a copy dataframe to check how many invalid lines of data have been removed
new_grades_df = grades.copy()
#Loop over the 'grades' data, check if length of id is not 9 and the elements of each iteration is not 26. If yes, print then remove.
for grade in new_grades_df: 
            value = grade.split(",")
            length = len(value)
            if length != 26:
                corrupted_ID = value[0]
                print("Invalid line of data:", corrupted_ID, " does not contain exactly 26 values:")
                print(grade)
                new_grades_df.remove(grade)
            id = value[0]
            id_length = len(id)
            if id_length != 9:
                corrupted_ID = value[0]
                print("Invalid line of data:", corrupted_ID, " is invalid")
                new_grades_df.remove(grade)
values_false = len(grades) - len(new_grades_df)
print("Total valid lines of data:", len(new_grades_df))
print("Total invalid lines of data:", values_false)


Total valid lines of data: 100
Total invalid lines of data: 0


Task 3: Calculate Exam Test:
How many high score 
average score
highest score
lowest score 
range of score(highest minus lowest) 
question most people skip(question num - how many)
question most answer incorrectly(question num - how many)

----------
Student of highscore 
Print result out
!!!!

In [12]:
answer_keys = "B,A,D,D,C,B,D,A,C,C,D,B,A,B,A,C,B,D,A,C,A,A,B,D,D"
answer_key = answer_keys.split(",")
answer_key_table = pd.DataFrame(answer_key)
answers_checked = []
score_list = []
highScore_loop = []
for grade in new_grades_df:
        value = grade.split(",")
        id = value[0]
        #Create "answer_table" data frame from the grades dataframe and remove student id from data frame
        answer_table = pd.DataFrame(value)
        answer_table = answer_table.iloc[1: , :]
        answer_table = answer_table.reset_index(drop=True)
        #Remove \n from value
        cVal = answer_table.iloc[24][0]
        cValSplit = cVal.split("\n")
        c = cValSplit[0]
        answer_table = answer_table.replace([cVal],c)
        #Compare student answers with answer keys to a new column called "checkAnswer", return True if correct, return the correct answer if false and "" if blank
        conditions  = [answer_table[0] == answer_key_table[0], answer_table[0] == "" , answer_table[0] != answer_key_table[0]]
        choices     = [ "True", '',answer_key_table[0]]
        answer_table["checkAnswer"] = np.select(conditions, choices, default=np.nan)
        #Calculate the quantity of correct, false and blank answers.
        num_correct = (answer_table['checkAnswer']=='True').sum()
        num_blank = (answer_table['checkAnswer']=='').sum()
        num_incorrect = len(answer_table[answer_table['checkAnswer'].map(lambda x: x !='True' and x !='')])
        #Calculate the score
        score = num_correct*4 + num_incorrect*(-1)
        #Make a new dataframe with 2 columns, id and score, then export the dataframe outside loop
        score_df = pd.DataFrame({'id': [id], 'score': [score]})
        score_list.append(score_df)
        #Get the list of student with high score to an instanced dataframe
        #highScore= sum(score_df[score_df['score'] > 80])

        #highScore_loop.append(highScore_list)
        #print(score_df)
    


        answers_checked.append(answer_table)
#print(highScore_loop)
score_pd = pd.concat(score_list)
#print(score_pd)
# Calculate max,min, mean, range of score, and median
mean = round(score_pd['score'].mean(),2)
max = score_pd['score'].max()
min = score_pd['score'].min()
RoS = max - min
num_students = score_pd['id'].nunique()
totalScore = score_pd['score'].sum()
med = round(totalScore/num_students,2)
print("Mean (average) score:", mean)
print("Highest score:", max)
print("Lowest score:", min)
print("Range of scores:", RoS)
print("Median score:", med)





answer_pd = pd.concat(answers_checked)
answer_pd.columns = ['studentAnswer', 'checkAnswer']
answer_pd['question'] = answer_pd.index


#Find out the question students most answer incorrectly and how many time the question is incorrect.
answers_incorrect_pd = answer_pd[answer_pd['checkAnswer'] != "True"]
answers_incorrect_pd = answers_incorrect_pd[answers_incorrect_pd['studentAnswer'] != ""]
answer_incorrect_count = answers_incorrect_pd.groupby('question')['question'].count()
question_most_incorrect = answer_incorrect_count.idxmax()
num_answer_incorrect = answer_incorrect_count.max()
ratio_incorrect = round(num_answer_incorrect / num_students,2)
print("Question that most people answer incorrectly:", question_most_incorrect)
print("Question", question_most_incorrect, "is answered incorrectly", num_answer_incorrect, "times","-",ratio_incorrect)
print("----------------------")

#Find out the question students most leave blank and how many time the question is leave blank.
answers_blank_pd = answer_pd[answer_pd['studentAnswer'] == ""]
answers_blank_count = answers_blank_pd.groupby('question')['question'].count()
question_most_leave_blank = answers_blank_count.idxmax()
num_answer_blank = answers_blank_count.max()
ratio_blank = round(num_answer_blank/ num_students,2)
print("Question that most people skip:", question_most_leave_blank)
print("Question", question_most_leave_blank, "is skipped", num_answer_blank, "times","-",ratio_blank)






Mean (average) score: 76.24
Highest score: 96
Lowest score: 56
Range of scores: 40
Median score: 76.24
Question that most people answer incorrectly: 17
Question 17 is answered incorrectly 17 times - 0.17
----------------------
Question that most people skip: 3
Question 3 is skipped 15 times - 0.15


Task 4: Export text file: 
as xxx_grades.txt
list as 'id', 'score'